In [1]:
import numpy as np
from astropy.table import Table

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

POPSYCLE SIM

In [2]:
pops = Table.read("data/research_paper/popsycle_sim.fits").to_pandas()

In [19]:
arr = []
for i in range(len(pops)):
    if (pops["rem_id_L"][i] == 103 and (pops["mass_L"][i] > 2.5 and pops["mass_L"][i] < 6)): #.5 for added padding to mass gap
        arr.append(i) #lower mass gap of bhs is between 2.5 or so and 5-7

OB110462 - Mass gap classified data

In [ ]:
ob = Table.read("data/research_paper/OB110462_DW_post.fits").to_pandas() #color data
ob.head()

In [26]:
ob.columns

Index(['logLike', 't0', 'u0_amp', 'tE', 'log10_thetaE', 'piS', 'piE_E',
       'piE_N', 'xS0_E', 'xS0_N', 'muS_E', 'muS_N', 'mL', 'piL', 'piRel',
       'muL_E', 'muL_N', 'muRel_E', 'muRel_N'],
      dtype='object')

combine data

In [54]:
popsCols = list(pops.columns)
obCols = list(ob.columns)
len(pops) + len(ob)

23139

In [52]:
wanted_ob = ["t0","u0_amp","tE","log10_thetaE","piE_E","muS_E","mL","piRel","muL_E","muRel_E"]
wanted_pops = ["t0","u0","t_E","theta_E","pi_E","mu_b_S","mass_L","pi_rel","mu_b_L","mu_rel"]

ob_to_pops = {"t0":"t0", "u0_amp":"u0","tE":"t_E","log10_thetaE":"theta_E","piE_E":"pi_E","muS_E":"mu_b_S","mL":"mass_L","piRel":"pi_rel","muL_E":"mu_b_L","muRel_E":"mu_rel"}


def gk(val,d=ob_to_pops):
    keys = [k for k, v in d.items() if v == val]
    if keys:
        return keys[0]
    return None


dfWhole = pd.DataFrame()
for j in wanted_ob:
    for k in range(len(ob[j])):
        dfWhole.at[k,j] = ob[j][k] #0,x 

dict = {"id":[]}

for j in wanted_pops:
    jVal = gk(j)
    dict[jVal] = pops[j]

for i in range(len(pops["rem_id_L"])):
    dict["id"].append(pops["rem_id_L"][i])

dfTotal = pd.concat([dfWhole,pd.DataFrame(dict)])


In [55]:
dfTotal

,t0,u0_amp,tE,log10_thetaE,piE_E,muS_E,mL,piRel,muL_E,muRel_E,id
0,55763.312017,-0.051724,287.263116,0.728651,0.021247,0.066662,6.890218,0.510787,-1.449263,1.515925,NaN
1,55763.143207,-0.056239,273.684662,0.363211,0.019904,0.122164,2.534837,0.258013,-0.426201,0.548365,NaN
2,55762.869349,-0.059092,280.197751,0.597241,0.019986,0.031534,4.004285,0.479873,-0.818066,0.849601,NaN
3,55764.431058,-0.042804,279.152526,0.617667,0.025575,0.126121,5.574564,0.378698,-1.393074,1.519195,NaN
4,55762.025369,-0.067009,273.444603,0.572748,0.020906,0.060268,3.557838,0.482480,-0.748844,0.809112,NaN
...,...,...,...,...,...,...,...,...,...,...,...
19713,-100.757093,-0.002355,10.272065,0.156485,0.026858,-2.123513,0.717312,0.004203,3.368475,5.564230,101.0
19714,44.697865,0.031002,44.315966,0.413579,0.080408,-0.152435,0.633246,0.033255,3.079775,3.408702,101.0
19715,36.049509,-0.020305,41.374829,0.282264,0.060556,-0.406050,0.573866,0.017093,-1.207997,2.491783,101.0
19716,107.731479,-0.341648,50.540982,0.495956,0.070537,-0.167646,0.865635,0.034983,1.031318,3.584180,101.0
